In [6]:
import mtt
from os.path import join, realpath
from os import getcwd
from pprint import pprint
sbfile = join(realpath(getcwd()),'kinase_cascade_stage7.sb')
wiring = mtt.Wiring.fromFile2(sbfile, {}, 1.)
wiring.blocks[1].ratC = 1.
wiring.connect(wiring.blocks[1].rv_up, wiring.blocks[0].Cprod)
wiring.connect(wiring.blocks[1].rv_up, wiring.blocks[0].Cprod)
wiring

In [7]:
model = mtt.MTT(wiring)
model.draw(3)

# Digitized current config

In [8]:
for block in wiring.blocks:
    print(block.getDigitizedParameterString(model))

# Simulation

In [9]:
# hardware simulation
model.reset()
r = model.simulate(0,10,1000)
for k in range(len(model.blocks)):
    print('block',k,'rate:',model.blocks[k].rate)
values = r.getLatestValues()
pprint(values)
model.plot()

# SBML Simulation

In [10]:
# SBML comparison
import tellurium as te
with open(join(realpath(getcwd()),'kinase_cascade_stage7.sb')) as f:
    rr_model = te.loada(f.read())
rr_model.selections = ['time','__s0','__s1','__s6','__s4','__s7','__s8','__s2','__s9']
rr_model.simulate(0,10,100)
for id, rate in zip(rr_model.getReactionIds(), rr_model.getReactionRates()):
    print(id,rate)
rr_model.plot()

for var,value in values.items():
    print('diff for {}: {}'.format(var,rr_model[var]-value))
    rr_model[var] = value
    
# for id, rate in zip(rr_model.getReactionIds(), rr_model.getReactionRates()):
#     print(id,rate)
# for k in range(len(model.blocks)):
#     print('block',k,'rate:',model.blocks[k].rate)